<a href="https://colab.research.google.com/github/Abhi23run/CSE8803_DLT_Project/blob/main/CodeBase/llama2_13b_chat_qlora_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install transformers==4.31
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install evaluate
!pip install -qqq trl==0.7.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 13.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB

In [2]:
%%capture
!pip install torch

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/CSE8803_DLT_Project/

/content/drive/MyDrive/CSE8803_DLT_Project


In [5]:
%ls

ashish_test/  finqa_dataset/          merged_model/  peft-dialogue-summary/
CodeBase/     llama2-docsum-adapter/  Papers/        README.md


In [7]:
dir_path = '/content/drive/MyDrive/finqa_dataset'
import os
os.chdir(dir_path)

In [8]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import transformers
from torch.optim import Adam
from tqdm import tqdm
from utils import *
import json
import pandas as pd
import torch
# from torch.utils.data import Dataset, DataLoader
import time
import evaluate
from datasets import Dataset, load_dataset
import random

In [9]:
with open(f"{dir_path}/train.json") as input_file:
        train_data = json.load(input_file)

with open(f"{dir_path}/dev.json") as input_file:
        valid_data = json.load(input_file)

with open(f"{dir_path}/test.json") as input_file:
        test_data = json.load(input_file)

In [10]:
assert len(train_data) == 6251, "The train data seems to be off, please check"
assert len(valid_data) == 883, "The validation data seems to be off, please check"
assert len(test_data) == 1147, "The test data seems to be off, please check"

**Creating a joint file for meaningful Table Serialization**

In [54]:
concatenated_df = pd.concat([pd.DataFrame(data)[['pre_text','post_text','filename', 'table_ori','table','id']].assign(dataset=name) for name, data in {'train': train_data, 'valid': valid_data, 'test': test_data}.items()], ignore_index=True)


In [56]:
concatenated_df.shape

(8281, 7)

In [61]:
concatenated_df.to_csv('/content/drive/MyDrive/finqa_dataset/df_combined_table_ser.csv',index=False)

File written to csv

In [11]:
train_data[0].keys()

dict_keys(['pre_text', 'post_text', 'filename', 'table_ori', 'table', 'qa', 'id', 'table_retrieved', 'text_retrieved', 'table_retrieved_all', 'text_retrieved_all'])

In [12]:
train_data[0]['id']

'ADI/2009/page_49.pdf-1'

In [43]:
train_data[2]['table_ori']

[['Year',
  'Gallons',
  'Average Priceper Gallon',
  'Aircraft FuelExpense',
  'Percent of TotalOperating Expenses'],
 ['2018', '4,447', '$2.23', '$9,896', '23.6%'],
 ['2017', '4,352', '1.73', '7,510', '19.6%'],
 ['2016', '4,347', '1.42', '6,180', '17.6%']]

In [44]:
train_data[12]['table']

[['', '2013'],
 ['labor-related deemed claim ( 1 )', '$ 1733'],
 ['aircraft and facility financing renegotiations and rejections ( 2 ) ( 3 )',
  '325'],
 ['fair value of conversion discount ( 4 )', '218'],
 ['professional fees', '199'],
 ['other', '180'],
 ['total reorganization items net', '$ 2655']]

In [ ]:
train_data[0]

In [21]:
def prepare_data(json_data, verbose=True):
    net=[]
    err_cnt=0
    for example in json_data:
        question = example["qa"]["question"]
        table = example["table"]
        id=example["id"]
        table_text = ""
        for row in table[1:]:
            this_sent = table_row_to_text(table[0], row)
            table_text += this_sent
        try:
            steps_text = format_steps(example["qa"]["steps"])
            inputs = {"id":id, "context": table_text, "question": question, "answer": steps_text}
            net.append(inputs)
        except:
            err_cnt+=1
            if verbose:
                print ("-"*25)
                print (example["filename"])
                print (example["qa"]["steps"])
                print ("-"*25+"\n")
    if err_cnt>0:
        print ("Net Errors:",err_cnt)
    return net

In [22]:
## Preparing huggingface dataset
data_splits = {'train': train_data, 'valid': valid_data, 'test': test_data}
datasets = {split: Dataset.from_pandas(pd.DataFrame(prepare_data(data, False))) for split, data in data_splits.items()}

Net Errors: 198
Net Errors: 35
Net Errors: 39


In [23]:
datasets

{'train': Dataset({
     features: ['id', 'context', 'question', 'answer'],
     num_rows: 6053
 }),
 'valid': Dataset({
     features: ['id', 'context', 'question', 'answer'],
     num_rows: 848
 }),
 'test': Dataset({
     features: ['id', 'context', 'question', 'answer'],
     num_rows: 1108
 })}

In [24]:
index=2
datasets['train'][index]

{'id': 'AAL/2018/page_13.pdf-2',
 'context': 'year the 2018 of gallons is 4447 ; the 2018 of average priceper gallon is $ 2.23 ; the 2018 of aircraft fuelexpense is $ 9896 ; the 2018 of percent of totaloperating expenses is 23.6% ( 23.6 % ) ;year the 2017 of gallons is 4352 ; the 2017 of average priceper gallon is 1.73 ; the 2017 of aircraft fuelexpense is 7510 ; the 2017 of percent of totaloperating expenses is 19.6% ( 19.6 % ) ;year the 2016 of gallons is 4347 ; the 2016 of average priceper gallon is 1.42 ; the 2016 of aircraft fuelexpense is 6180 ; the 2016 of percent of totaloperating expenses is 17.6% ( 17.6 % ) ;',
 'question': 'what was the total operating expenses in 2018 in millions',
 'answer': 'Step 1: Divide 9896 by 23.6%. This gives the result: 41932'}

**Loading in Llama7B-Chat mode**

In [25]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [26]:
import getpass

# Prompt for the Hugging Face token
hf_token = getpass.getpass("Enter your Hugging Face token: ")

# Use the token in your application
# For example, setting an environment variable (optional)
import os
os.environ['HUGGINGFACE_TOKEN'] = hf_token
# Now you can use Hugging Face APIs with the token
# Example: loading a model, accessing API, etc.


Enter your Hugging Face token: ··········


In [27]:
# model_id =  "NousResearch/Llama-2-7b-hf"
model_id = "meta-llama/Llama-2-13b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto",use_auth_token=hf_token)

tokenizer = AutoTokenizer.from_pretrained(model_id,use_auth_token=hf_token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [28]:
index=2
datasets['train'][index]

{'id': 'AAL/2018/page_13.pdf-2',
 'context': 'year the 2018 of gallons is 4447 ; the 2018 of average priceper gallon is $ 2.23 ; the 2018 of aircraft fuelexpense is $ 9896 ; the 2018 of percent of totaloperating expenses is 23.6% ( 23.6 % ) ;year the 2017 of gallons is 4352 ; the 2017 of average priceper gallon is 1.73 ; the 2017 of aircraft fuelexpense is 7510 ; the 2017 of percent of totaloperating expenses is 19.6% ( 19.6 % ) ;year the 2016 of gallons is 4347 ; the 2016 of average priceper gallon is 1.42 ; the 2016 of aircraft fuelexpense is 6180 ; the 2016 of percent of totaloperating expenses is 17.6% ( 17.6 % ) ;',
 'question': 'what was the total operating expenses in 2018 in millions',
 'answer': 'Step 1: Divide 9896 by 23.6%. This gives the result: 41932'}

In [29]:
def format_instruction(context: str, question: str, answer: str):
	return f"""### Instruction:
Answer the question using the information from the context below. Think step by step and solve the problem.

### Context:
{context.strip()}

### Question:
{question.strip()}

### Answer:
{answer}
""".strip()

In [30]:
def generate_instruction_dataset(data_point):

    return {
        "context": data_point["context"],
        "question": data_point["question"],
        "answer": data_point["answer"],
        "input_prompt": format_instruction(data_point["context"],data_point["question"],data_point["answer"])
    }

In [31]:
def process_dataset(data: Dataset):
    return (
        data.shuffle(seed=42)
        .map(generate_instruction_dataset)
    )

In [32]:
## APPLYING PREPROCESSING ON WHOLE DATASET
datasets["train"] = process_dataset(datasets["train"])
datasets["test"] = process_dataset(datasets["test"])
datasets["valid"] = process_dataset(datasets["valid"])

Map:   0%|          | 0/6053 [00:00<?, ? examples/s]

Map:   0%|          | 0/1108 [00:00<?, ? examples/s]

Map:   0%|          | 0/848 [00:00<?, ? examples/s]

In [33]:
index=0
print(datasets["train"]["input_prompt"][index])

### Instruction:
Answer the question using the information from the context below. Think step by step and solve the problem.

### Context:
reporting unit the retail brokerage of december 31 2012 goodwill is $ 1791.8 ; the retail brokerage of december 31 2012 % ( % ) of fair value to book value is 190% ( 190 % ) ;reporting unit the market making of december 31 2012 goodwill is 142.4 ; the market making of december 31 2012 % ( % ) of fair value to book value is 115% ( 115 % ) ;reporting unit the total goodwill of december 31 2012 goodwill is $ 1934.2 ; the total goodwill of december 31 2012 % ( % ) of fair value to book value is ;

### Question:
what percentage of total goodwill is comprised of market making at december 31 2012?

### Answer:
Step 1: Divide 142.4 by 1934.2. This gives the result: 7%


In [49]:
import warnings
warnings.filterwarnings('ignore')
# index = 4
index = 1

context = datasets['train'][index]['context']
answer = datasets['train'][index]['answer']
question = datasets['train'][index]['question']

prompt = f"""
Answer the question using the information from the context below. Think step by step and solve the problems in concise steps.

### Context:
{context}

### Question:
{question}

### Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        temperature=0.2,
        do_sample=True,
        max_new_tokens=1024,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE ANSWER:\n{answer}\n')
print(dash_line)
print(f'MODEL GENERATED ANSWER - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Answer the question using the information from the context below. Think step by step and solve the problem.

### Context:
the vendor non-trade receivables of 2007 is $ 2392 ; the vendor non-trade receivables of 2006 is $ 1593 ;the nand flash memory prepayments of 2007 is 417 ; the nand flash memory prepayments of 2006 is 208 ;the other current assets of 2007 is 996 ; the other current assets of 2006 is 469 ;the total other current assets of 2007 is $ 3805 ; the total other current assets of 2006 is $ 2270 ;

### Question:
what percentage of total other current assets in 2006 was comprised of nand flash memory prepayments?

### Answer:

---------------------------------------------------------------------------------------------------
BASELINE ANSWER:
Step 1: Divide 208 by 2270. This gives the result: 9%

---------------------------------------------------------------------

In [50]:
answer

'Step 1: Divide 208 by 2270. This gives the result: 9%'

In [47]:
context = 'There are 2 apples and 3 oranges in a basket. I bought a dozen of apples today.'
answer = ''
question = 'What is the total number of fruits that I have?'

prompt = f"""
Answer the question using the information from the context below. Think step by step and generate the final answer.

### Context:
{context}

### Question:
{question}

### Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        temperature=0.2,
        do_sample=True,
        max_new_tokens=1024,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE ANSWER:\n{answer}\n')
print(dash_line)
print(f'MODEL GENERATED ANSWER - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Answer the question using the information from the context below. Think step by step and generate the final answer.

### Context:
There are 2 apples and 3 oranges in a basket. I bought a dozen of apples today.

### Question:
What is the total number of fruits that I have?

### Answer:

---------------------------------------------------------------------------------------------------
BASELINE ANSWER:


---------------------------------------------------------------------------------------------------
MODEL GENERATED ANSWER - ZERO SHOT:

Answer the question using the information from the context below. Think step by step and generate the final answer.

### Context:
There are 2 apples and 3 oranges in a basket. I bought a dozen of apples today.

### Question:
What is the total number of fruits that I have?

### Answer:

Step 1:
I bought a dozen of apples, so I have 12 apples

In [50]:
table = ['''[['Year',
  'Gallons',
  'Average Priceper Gallon',
  'Aircraft FuelExpense',
  'Percent of TotalOperating Expenses'],
 ['2018', '4,447', '$2.23', '$9,896', '23.6%'],
 ['2017', '4,352', '1.73', '7,510', '19.6%'],
 ['2016', '4,347', '1.42', '6,180', '17.6%']]
''','''[['(In Millions)', 'Dec 28,2013', 'Dec 29,2012'],
 ['Available-for-sale investments', '$18,086', '$14,001'],
 ['Cash', '854', '593'],
 ['Equity method investments', '1,038', '992'],
 ['Loans receivable', '1,072', '979'],
 ['Non-marketable cost method investments', '1,270', '1,202'],
 ['Reverse repurchase agreements', '800', '2,850'],
 ['Trading assets', '8,441', '5,685'],
 ['Total cash and investments', '$31,561', '$26,302']]''']
instruction = ['''Given a table as a 2D list above, understand the semantics and structure through the values present. Produce a serialized text that lists all the information present in the table in a concise manner.''']*2 + ['''Follow the pattern above through the examples given and produce a serialized text that lists all the information present in the table in a concise manner.''']

answers=['''As of Dec 28, 2013, the available-for-sale investments were valued at $18,086 million, cash reserves were at $854 million, equity method investments stood at $1,038 million, loans receivable were at $1,072 million, non-marketable cost method investments were valued at $1,270 million, reverse repurchase agreements totaled $800 million, and trading assets were at $8,441 million. This resulted in a total cash and investments figure of $31,561 million. In comparison, on Dec 29, 2012, available-for-sale investments were $14,001 million, cash at $593 million, equity method investments at $992 million, loans receivable at $979 million, non-marketable cost method investments at $1,202 million, reverse repurchase agreements at $2,850 million, and trading assets at $5,685 million, with a total cash and investments of $26,302 million.''',\
         '''In 2018, 4,447 gallons of fuel were used at an average price per gallon of $2.23, resulting in an aircraft fuel expense of $9,896, which constituted 23.6% of the total operating expenses. In 2017, the fuel usage was 4,352 gallons with an average price of $1.73 per gallon, leading to an aircraft fuel expense of $7,510 and accounting for 19.6% of the total operating expenses. In 2016, a total of 4,347 gallons were used, the average price per gallon was $1.42, the aircraft fuel expense amounted to $6,180, and it represented 17.6% of the total operating expenses.''']

prompt = f"""
### Table:
[['In millions', 'InterestOnlyProduct', 'PrincipalandInterestProduct'],
 ['2013', '$1,338', '$221'],
 ['2014', '2,048', '475'],
 ['2015', '2,024', '654'],
 ['2016', '1,571', '504'],
 ['2017', '3,075', '697'],
 ['2018 and thereafter', '5,497', '4,825'],
 ['Total (a)', '$15,553', '$7,376']]

### Instruction:
{instruction[0]}

### Answer:
"""
inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"].to('cuda'),
        temperature=0.2,
        do_sample=True,
        max_new_tokens=1024,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
# print(dash_line)
# print(f'INPUT PROMPT:\n{prompt}')
# print(dash_line)
# print(f'BASELINE ANSWER:\n{answer}\n')
# print(dash_line)
print(f'MODEL GENERATED ANSWER - ZERO SHOT:\n{output}')

MODEL GENERATED ANSWER - ZERO SHOT:

### Table:
[['In millions', 'InterestOnlyProduct', 'PrincipalandInterestProduct'],
 ['2013', '$1,338', '$221'],
 ['2014', '2,048', '475'],
 ['2015', '2,024', '654'],
 ['2016', '1,571', '504'],
 ['2017', '3,075', '697'],
 ['2018 and thereafter', '5,497', '4,825'],
 ['Total (a)', '$15,553', '$7,376']]

### Instruction:
Given a table as a 2D list above, understand the semantics and structure through the values present.Produce a serialized text that lists all the information present in the table in a concise manner.

### Answer:
The table above contains information about the interest-only and principal and interest products for different years. The table has the following columns:

* 'In millions': This column lists the amount in millions for each year.
* 'InterestOnlyProduct': This column lists the interest only product for each year.
* 'PrincipalandInterestProduct': This column lists the principal and interest product for each year.

The table has the

In [ ]:
table='''[['In millions', 'InterestOnlyProduct', 'PrincipalandInterestProduct'],
 ['2013', '$1,338', '$221'],
 ['2014', '2,048', '475'],
 ['2015', '2,024', '654'],
 ['2016', '1,571', '504'],
 ['2017', '3,075', '697'],
 ['2018 and thereafter', '5,497', '4,825'],
 ['Total (a)', '$15,553', '$7,376']]'''

instruction = '''Given a table as a 2D list above, understand the semantics and structure through the values present. Produce a serialized text that lists all the information present in the table in a concise manner.'''
prompt = f"""
### Table:{table}

### Instruction:
{instruction}

### Answer:
"""